In [1]:
from keras.preprocessing.image import ImageDataGenerator

import csv
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

import itertools
from astropy.io import fits

import os
import sys

CLASS_NUM = 2 # the number of classes for classification

#img_channels = 1
img_channels = 4
IMG_CHANNEL = 4
IMG_SIZE = 50

#input_shape = (1, 239, 239) # ( channels, cols, rows )
raw_size = (239, 239, img_channels)
#raw_size = (48, 48, img_channels)
input_shape = (50, 50, IMG_CHANNEL)
#input_shape = (24, 24, img_channels)

train_test_split_rate = 0.8
#train_test_split_rate = 1
nb_epoch = 20
batch_size = 10
validation_split = 0.1
#validation_split = 0.0


BATCH_SIZE = 10
NEPOCH = 100
KFOLD = 5

IMG_IDX = 2
LABEL_IDX = 2 +  IMG_CHANNEL
PNG_LABEL_IDX = 2 + IMG_CHANNEL

FILE_HOME = "/Users/sheep/Documents/research/project/hsc"

DATA_ROOT_DIR = '/Users/sheep/Documents/research/project/hsc'
ROOT_DIR = os.path.dirname(os.path.abspath(sys.argv[0]))

PNG_IMG_DIR = '/Users/sheep/Documents/research/project/hsc/png_images'
PNG_IMG_DIR = '/Users/sheep/documents/research/project/hsc/png_images'

SAVE_DIR = '/Users/sheep/Documents/research/project/hsc/saved_data'
DATASET = 'dataset/dropout_test.csv'


class DatasetLoader:

    def __init__(self, csv_file_path, root_dir, start=1, end=12266):
        data_frame = pd.read_csv(csv_file_path, header=None)
        self.root_dir = root_dir
        self.dataset_frame_list = []
        self.dataset = []
        for i in range(CLASS_NUM):
            if i == 1:
                tmp_dataframe = data_frame[data_frame[LABEL_IDX]==i]
                self.dataset_frame_list.append(tmp_dataframe[start:end])
            else:
                self.dataset_frame_list.append(data_frame[data_frame[LABEL_IDX]==i])
            self.dataset.append( self.create_dataset(i) )

    def create_dataset(self, label):
        data_frame = self.get_dataframe(label)
        data_list = []

        for idx, row_data in data_frame.iterrows():
            img_no = str(row_data[0])

            png_img_name = row_data[1]

            img_names = row_data[2:IMG_IDX+IMG_CHANNEL]
            img_names = [ path for path in img_names ]

            label = row_data[LABEL_IDX]
            #label = np_utils.to_categorical(label, num_classes=CLASS_NUM)

            image = self.load_image(img_names)
            image = self.crop_center(image, IMG_SIZE, IMG_SIZE)
            #image = self.normalize2(image)
            #image_feature = self.extract_feature(( normalized_image + 0.5 )*10)
            #print(image_feature, label)

            data_list.append( (label, image, img_no, png_img_name, img_names) )

        return data_list
    
    
    def normalize1(self, image):
        flat_image = image.flatten()
        mean = flat_image.mean()
        std = flat_image.std()
        image = np.where(image < mean - 1*std, mean - 1*std, image)
        image = np.where(image > mean + 1*std, mean + 1*std, image)
        image = (image + 1*std - mean)
        
        return image
    
    def normalize2(self, image):
        image = image + 1.0
        image = np.where(image < 0, 0, image)
        image = np.where(image  > 5, 5, image)
        return image

        
    def crop_center(self, img,cropx, cropy):
        y,x,z = img.shape
        startx = x//2-(cropx//2)
        starty = y//2-(cropy//2)
        return img[starty:starty+cropy,startx:startx+cropx,:]

    def get_dataframe(self, label):
        return self.dataset_frame_list[label]

    def get_dataset(self, label):
        return self.dataset[label]

    def zoom_img(self, img, original_size, pickup_size):
        startpos = int(original_size / 2) - int(pickup_size / 2)
        img = img[startpos:startpos+pickup_size, startpos:startpos+pickup_size]
        return img

    def load_image(self, img_paths):
        image_path_list = [self.root_dir + img_path for img_path in img_paths]
        image_list = []
        for filepath in image_path_list:
            row_data = fits.getdata(filepath)
            image_list.append(row_data)
        image = np.array([img for img in image_list]).transpose(1,2,0)
        return image



#create dataset for cross validation
print('data read start')

dataset_no_normalize = DatasetLoader(DATASET, DATA_ROOT_DIR, 1, 5000)
other_dataset_no_normalize = DatasetLoader(DATASET, DATA_ROOT_DIR, start=5001)

print('finished')


Using TensorFlow backend.
/Users/sheep/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


data read start
finished


In [10]:
def normalize1(image):
    flat_image = image.flatten()
    mean = flat_image.mean()
    std = flat_image.std()
    image = np.where(image < mean - 2*std, mean - 2*std, image)
    image = np.where(image > mean + 2*std, mean + 2*std, image)
    image = (image + 2*std - mean)

    return image

def normalize2(image):
    image = image + 1.0
    image = np.where(image < 0, 0, image)
    image = np.where(image  > 5, 5, image)
    return image

def normalize_all_data(dataset_obj):
    for data_list in dataset_obj.dataset:
        for idx in range(len(data_list)):
            (label, image, img_no, png_img_name, img_names) = data_list[idx]
            data_list[idx] =  (label, normalize1(image), img_no, png_img_name, img_names)

def normalize_all_data2(dataset_obj):
    for idx in range(len(dataset_obj)):
        (label, image, img_no, png_img_name, img_names) = dataset_obj[idx]
        dataset_obj[idx] =  (label, DatasetLoader.normalize2(None, image), img_no, png_img_name, img_names)
            
print('normalize start')
dataset = dataset_no_normalize
other_dataset = other_dataset_no_normalize
normalize_all_data(dataset)
true_dataset = dataset.get_dataset(1)
false_dataset = dataset.get_dataset(0)

normalize_all_data(other_dataset)
other_true_dataset = other_dataset.get_dataset(1)
other_true_test_img = list(map(lambda data: data[1], other_true_dataset))
other_true_test_label = list(map(lambda data: data[0], other_true_dataset))
other_true_test_catalog_ids_set = list(map(lambda data: data[2], other_true_dataset))
other_true_test_png_img_set = list(map(lambda data: data[3], other_true_dataset))
other_true_test_paths_set = list(map(lambda data: data[4], other_true_dataset))
print('normalize finished')

normalize start
normalize finished


In [11]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import itertools
IMG_CHANNEL = 4

FEATURE_RANGE = [15]
PATTERNS = list(itertools.combinations([i for i in range(IMG_CHANNEL)], 2))

def extract_feature(image):
    feature_list = []
    image = image/255
    for size in FEATURE_RANGE:
        for (channel1, channel2) in PATTERNS:
            image1 = image[:,:,channel1]
            image2 = image[:,:,channel2]
            feature_list.append(calc_pixel_ratio(image1, image2, size))
            feature_list.append(calc_pixel_ratio_exp(image1, image2, size))

        for channel in range(IMG_CHANNEL):
            cropped_image = image[:,:,channel]
            mean, std = calc_pixel_mean(cropped_image, size)
            feature_list.append(mean)
            feature_list.append(std*std)
            
    return np.array( feature_list )

def calc_pixel_mean(image, size):
    cropped_image = crop_center2D(image, size, size)
    return cropped_image.mean(), cropped_image.std()

def calc_pixel_ratio(image1, image2, size):
    cropped_image1 = crop_center2D(image1, size, size)
    cropped_image2 = crop_center2D(image2, size, size)

    return cropped_image1.sum()/cropped_image2.sum()

def calc_pixel_ratio_exp(image1, image2, size):
    cropped_image1 = crop_center2D(image1, size, size)
    cropped_image2 = crop_center2D(image2, size, size)

    return np.exp(cropped_image1).sum()/np.exp(cropped_image2).sum()

def crop_center2D(img,cropx, cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)
    return img[starty:starty+cropy,startx:startx+cropx]

def write_result(zipped_result, output_path):
    with open(output_path, 'w') as f:
        writer = csv.writer(f)
        for result in zipped_result:
            cat_id = result[0]
            img_paths = result[2]
            combined_img_path = result[1]
            label = result[3]
            float_formatter = lambda x: "%.4f" % x
            pred = result[4]
            # row should be [cat_id, img1, img2, img3, combined_img, correct_label, [probabilties], answer]
            row = [cat_id, img_paths, combined_img_path, label, pred]
            writer.writerow(row)

print('start')
kfold = KFold(n_splits=5)

true_dataset_fold = kfold.split(true_dataset)
false_dataset_fold = kfold.split(false_dataset)

accuracies = []
for fold_idx, ( (true_train_idx, true_test_idx), (false_train_idx, false_test_idx) ) in\
        enumerate( zip(true_dataset_fold, false_dataset_fold) ):

    print(fold_idx)
    true_train_data = [ true_dataset[idx] for idx in true_train_idx]
    true_test_data = [ true_dataset[idx] for idx in true_test_idx ]
    false_train_data = [ false_dataset[idx] for idx in false_train_idx ]
    false_test_data = [ false_dataset[idx] for idx in false_test_idx ]

    #data augumentation
    datagen = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True)

    tmp_false_train_data = false_train_data
    false_train_data = []
    for idx, data in enumerate( tmp_false_train_data ):
        label = data[0]
        img = data[1]
        img_no = data[2]
        img_name = data[3]
        img_names = data[4]
        expanded_image = np.expand_dims(img, axis=0)
        generator = datagen.flow(expanded_image, batch_size=1, save_prefix='img', save_format='png')
        for ite in range(19):
            batch = generator.next()
            false_train_data.append( (label, batch[0], img_no, img_name, img_names) )

    true_train_img_feature = list(map(lambda data: extract_feature(data[1]), true_train_data))
    true_train_img = list(map(lambda data: data[1], true_train_data))
    true_train_label = list(map(lambda data: data[0], true_train_data))
    
    true_test_img_feature = list(map(lambda data: extract_feature(data[1]), true_test_data)) + [extract_feature(data) for data in other_true_test_img]
    true_test_img = list(map(lambda data: data[1], true_test_data)) + other_true_test_img
    true_test_label = list(map(lambda data: data[0], true_test_data)) + other_true_test_label
    true_test_catalog_ids_set = list(map(lambda data: data[2], true_test_data)) + other_true_test_catalog_ids_set
    true_test_png_img_set = list(map(lambda data: data[3], true_test_data)) + other_true_test_png_img_set
    true_test_paths_set = list(map(lambda data: data[4], true_test_data)) + other_true_test_paths_set
    
    false_train_img_feature = list(map(lambda data: extract_feature(data[1]), false_train_data))
    false_train_img = list(map(lambda data: data[1], false_train_data))
    false_train_label = list(map(lambda data: data[0], false_train_data))
    
    false_test_img_feature = list(map(lambda data: extract_feature(data[1]), false_test_data))
    false_test_img = list(map(lambda data: data[1], false_test_data))
    false_test_label = list(map(lambda data: data[0], false_test_data))
    false_test_catalog_ids_set = list(map(lambda data: data[2], false_test_data))
    false_test_png_img_set = list(map(lambda data: data[3], false_test_data))
    false_test_paths_set = list(map(lambda data: data[4], false_test_data))
    
    train_img_feature = true_train_img_feature + false_train_img_feature
    train_label = true_train_label + false_train_label
    
    test_img_feature = true_test_img_feature + false_test_img_feature
    test_label = true_test_label + false_test_label
    test_catalog_id_set = true_test_catalog_ids_set + false_test_catalog_ids_set
    test_png_img_set = true_test_png_img_set + false_test_png_img_set
    test_path_set = true_test_paths_set + false_test_paths_set
    
    print('true train', len(true_train_img))
    print('true test', len(true_test_img))
    print('false train', len(false_train_img))
    print('false test', len(false_test_img))

    #SVM classification
    model = RandomForestClassifier(criterion='gini', n_estimators=100, random_state=10)
    model.fit(train_img_feature, train_label)

    pred_result = model.predict(test_img_feature)
    write_result(zip(test_catalog_id_set, test_png_img_set, test_path_set, test_label, pred_result), 'res{}.csv'.format(fold_idx))
    
    print(metrics.accuracy_score(test_label, pred_result))
    print(metrics.confusion_matrix(test_label, pred_result))

start
0


/Users/sheep/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in float_scalars


true train 3999
true test 8265
false train 3990
false test 53


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').